# Quick analysis

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown
from utilities import redact_small_numbers


In [2]:
# import data

df = pd.read_feather("../output/input.feather")
#df = pd.read_csv("../output/input.csv")

# for binary fields, replace zeros with null so that `count` can
binary_fields = ['elective_short_stay', 'elective_or_op']
for f in binary_fields:
    df[f] = df[f].astype('int64').replace(0,np.nan)

maxdate = df["any_admission_date"].max()
display(Markdown(f"Latest admission date: {maxdate}"))
display(Markdown(f"Therapeutics: {df['outpatient_covid_therapeutic_name'].unique().tolist()}"))

Latest admission date: 2022-04-25

Therapeutics: ['paxlovid', 'molnupiravir', 'sotrovimab', nan, 'remdesivir', 'casirivimab and imdevimab']

In [3]:

fields = {
'Outpatient':
    ['outpatient_covid_therapeutic_date',
     'elective_or_op',
     'elective_short_stay', 
     'daycase_admission_date',
     'elective_x892_date', 
     'elective_x292_date', 
     'hospital_attendance_date'],
'Inpatient':      
    ['inpatient_covid_therapeutic_date',
     'any_admission_date', 
     'any_admission_x892_date',
     'any_admission_x292_date']
}

for x in fields:
    f = fields[x][0]
    display(Markdown(f"## {x}"))

    # filter to treatment dates within available SUS data range
    df1 = df.copy().loc[(df[f] <= maxdate)]

    if x=="Inpatient": # don't count admissions if discharge date was after treatment date
        display(Markdown(f"Note: for inpatients, recent spells may not yet have completed so some data may be missing"))
        for c in fields[x][1:]: # for each admission type
            # compare discharge date with treatment date and remove admission date if not in window
            df1.loc[df1[f] > df1[c.replace("admission", "discharge")], c] = np.nan
            
    # create df for mabs and separate df for Antivirals
    name_field = f'{x.lower()}_covid_therapeutic_name'
    df1[name_field] = df1[name_field].astype(str).fillna("")
    df_mab = df1.loc[df1[name_field].str.contains("mab")]
    df_non_mab = df1.loc[~df1[name_field].str.contains("mab")]
    treatments = {"MABs":df_mab,
                  "Antivirals":df_non_mab}

    # Breakdown by treatment type (MABs/Avs)
    for t in treatments:
        df_t = treatments[t]
        summary1 = pd.DataFrame(df_t[fields[x]].count()).rename(columns={0:"count"})
        summary1["percent"] = (100*(summary1["count"]/summary1["count"][f])).fillna(0).round(2).astype(int)
        summary1.index = summary1.index.str.replace("_date","")
        summary1 = redact_small_numbers(summary1, n=5, rate_column="percent")
        out_text = f"""Out of **{summary1["count"][0]}** patients treated in *{x}* settings with *{t}*, **{summary1["count"][1]} ({summary1["percent"][1]}%)** had *{summary1.index[1].replace("_"," ")}* in SUS"""

        display(Markdown(f"### {t}"), Markdown(out_text), summary1)


    # Breakdown by region and age group (MABs only, most inclusive hospital record field only)
    for col in ["region_covid_therapeutics","age_group"]:
        summary2 = pd.DataFrame(df_mab.groupby(col)\
                [fields[x][0:2]].count()).rename(columns={0:"count"})
        summary2["percent"] = (100*summary2[fields[x][1]]/summary2[f]).fillna(0).round(2).astype(int)
        summary2.columns = summary2.columns.str.replace("_date","")
        summary2 = redact_small_numbers(summary2, n=5, rate_column="percent")

        c_min = summary2["percent"].min()
        c_max = summary2["percent"].max()
        out_text = f"""The percent of MABs patients treated in *{x}* settings found in SUS varied by *{col}* from **{c_min}%** to **{c_max}%**"""
        display(Markdown("### Breakdown by region and age group (MABs only)"), Markdown(out_text), summary2)

## Outpatient

### MABs

Out of **278** patients treated in *Outpatient* settings with *MABs*, **140 (50.0%)** had *elective or op* in SUS

,count,percent
outpatient_covid_therapeutic,278,100.0
elective_or_op,140,50.0
elective_short_stay,203,73.0
daycase_admission,148,53.0
elective_x892,29,10.0
elective_x292,26,9.0
hospital_attendance,37,13.0


### Antivirals

...

The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *region_covid_therapeutics* from **32.0%** to **42.0%**

,inpatient_covid_therapeutic,any_admission,percent
region_covid_therapeutics,,,
East,28,NaN,NaN
East Midlands,28,10.0,35.0
London,54,21.0,38.0
North East,31,12.0,38.0
North West,30,12.0,40.0
South East,38,13.0,34.0
South West,25,8.0,32.0
West Midlands,27,11.0,40.0
Yorkshire and The Humber,21,9.0,42.0


### Breakdown by region and age group (MABs only)

The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *age_group* from **28.0%** to **55.0%**

,inpatient_covid_therapeutic,any_admission,percent
age_group,,,
12-24,33,13.0,39.0
25-34,21,NaN,NaN
35-44,54,17.0,31.0
45-54,45,13.0,28.0
55-64,34,11.0,32.0
65-74,35,14.0,40.0
75+,31,12.0,38.0
missing,29,16.0,55.0
